# 키움 로그인

In [1]:
import sys
from PyQt5.QtWidgets import QApplication
from kiwoom_api.api import Kiwoom, DataFeeder, Executor

In [2]:
app = QApplication(sys.argv)

kiwoom = Kiwoom() # Kiwoom 인스턴스 생성
kiwoom.commConnect() # API 접속
feeder = DataFeeder(kiwoom)

2020-06-12 14:11:40.827839 Connection Successful


# 키움 분봉 주식데이터 가져오기

In [3]:
import pandas as pd
import time

In [4]:
Kospi_codes = pd.read_csv('./KOSPI200.csv', header=None) #코스피200 목록 가져오기
Kospi_codes.columns = ['codes', 'name']

In [5]:
def min_request(trCode, n, **kwargs):   
    for k, v in kwargs.items():
        kiwoom.setInputValue(k, v)

    kiwoom.commRqData('주식분봉차트조회요청', trCode, n, "0000")
    return getattr(kiwoom, trCode)

In [6]:
def get_min_data(code, n): #n : 몇번 가져올 것인가? 1회 900개
    params = {"종목코드": code, "틱범위":1, "수정주가구분" : "1"}
    
    data = []
    last_date = None
    
    for i in range(n):
        time.sleep(0.3)

        if i == 0:
            tmp_data = min_request(trCode="OPT10080", n=0, **params)
        else:
            tmp_data = min_request(trCode="OPT10080", n=2, **params)
                
        if tmp_data['멀티데이터'][-1]['체결시간'] != last_date:
            print(last_date)
            data.append(tmp_data)
            last_date = tmp_data['멀티데이터'][-1]['체결시간']
        else:
            print(code, " 마지막 i=",i)
            break
        
    tmp = pd.DataFrame(data[0]['멀티데이터'])

    for i in range(1, len(data)):
        tmp = tmp.append(pd.DataFrame(data[i]['멀티데이터']))

    tmp = tmp.reset_index(drop=True).loc[:, ["체결시간", "현재가", "시가", "고가", "저가", "거래량"]]
        
    return tmp

In [7]:
for idx, code, name in Kospi_codes.iloc[:20,:].itertuples(): #코스피 상위 20개 목록의 주식을 가져온다.
    code = '%06d' % code
    print(idx, code, name)
    min_data = get_min_data(code, 200)
    min_data.to_csv('./min_data/{}_{}_.csv'.format(name, code)) #저장은 노트북파일과 동일한 폴더의 min_data에 저장한다.
    time.sleep(1)

0 005930 삼성전자


2020-06-12 14:14:31.770158  commRqData 주식분봉차트조회요청
{'BASC_DT': '2020-06-12',
 'EVENT': 'eventReceiveTrData',
 'REQUEST_NAME': '주식분봉차트조회요청',
 'TIME': '2020-06-12 14:14:34.072035',
 'TR_CODE': 'OPT10080'}


None


2020-06-12 14:14:34.489891  commRqData 주식분봉차트조회요청
{'BASC_DT': '2020-06-12',
 'EVENT': 'eventReceiveTrData',
 'REQUEST_NAME': '주식분봉차트조회요청',
 'TIME': '2020-06-12 14:14:34.813021',
 'TR_CODE': 'OPT10080'}


20200610115900


...

2020-06-12 14:14:36.002841  commRqData 주식분봉차트조회요청
{'BASC_DT': '2020-06-12',
 'EVENT': 'eventReceiveTrData',
 'REQUEST_NAME': '주식분봉차트조회요청',
 'TIME': '2020-06-12 14:14:36.362877',
 'TR_CODE': 'OPT10080'}


20200603134900


2020-06-12 14:14:36.779762  commRqData 주식분봉차트조회요청
{'BASC_DT': '2020-06-12',
 'EVENT': 'eventReceiveTrData',
 'REQUEST_NAME': '주식분봉차트조회요청',
 'TIME': '2020-06-12 14:14:37.127857',
 'TR_CODE': 'OPT10080'}


20200601113300


KeyboardInterrupt: 

In [ ]:
#2019-6-3일 9:00부터 가져온다.